## Limpieza de datos

Rara vez los archivos con los que trabajemos estarán listos a usar para modelar por lo que previamente necesitamos realizar una limpieza de los datos perdidos o dañados y reemplazarlos por otros valores o una representación de estos, asi como también vamos a tener que normalizar los datos por tener valores muy diferentes.

### Limpiando filas con NaNs

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('NaNDataset.csv')
df.isnull().sum() # contamos la cantidad de valores nulls o NaNs que se encuentran en nuestro dataFrame

A    0
B    2
C    0
dtype: int64

In [4]:
df

,A,B,C
0,1,2.0,3
1,4,NaN,6
2,7,NaN,9
3,10,11.0,12
4,13,14.0,15
5,16,17.0,18


Observamos cuando un dataFrame no tiene alguna información entonces pandas lo reemplaza por NaN.

### Reemplazando NaN con la media de la columna

In [5]:
df.B = df.B.fillna(df.B.mean())
df

,A,B,C
0,1,2.0,3
1,4,11.0,6
2,7,11.0,9
3,10,11.0,12
4,13,14.0,15
5,16,17.0,18


### Removiendo Filas con NaNs

In [12]:
df = pd.read_csv('NaNDataset.csv')
df = df.dropna()
df

,A,B,C
0,1,2.0,3
3,10,11.0,12
4,13,14.0,15
5,16,17.0,18


Como al eliminar las filas en donde se encontraban valores NaN los indices quedan variados, podemos resetear los indices de la siguiente manera:

In [13]:
df = df.reset_index(drop = True)
df

,A,B,C
0,1,2.0,3
1,10,11.0,12
2,13,14.0,15
3,16,17.0,18


Ponemos drop = True para no quedarnos con la columna con los indices variados, no poner un drop significa un drop = False por defecto.

### Removiendo Filas duplicadas

In [23]:
df = pd.read_csv('DuplicateRows.csv')
df

,A,B,C
0,1,2,3
1,4,5,6
2,4,5,6
3,7,8,9
4,7,18,9
5,10,11,12
6,10,11,12
7,13,14,15
8,16,17,18


#### Mi manera de eliminar duplicados

la function duplicated() nos retorna una Serie de booleanos en donde True indica que ese valor **ya** tiene una repetición en el data frame, luego como podemos tratar los dataframes por los valores (usando corchetes []), realizé esta manera de eliminar duplicados.

In [29]:
df.duplicated()

0    False
1    False
2     True
3    False
4    False
5    False
6     True
7    False
8    False
dtype: bool

In [28]:
df[df.duplicated()!=True]

,A,B,C
0,1,2,3
1,4,5,6
3,7,8,9
4,7,18,9
5,10,11,12
7,13,14,15
8,16,17,18


In [30]:
df[df.duplicated(keep=False)] 

,A,B,C
1,4,5,6
2,4,5,6
5,10,11,12
6,10,11,12


se muestra todos los valores que se repetin (todos).

In [32]:
df.drop_duplicates(keep='first', inplace = True)
df

,A,B,C
0,1,2,3
1,4,5,6
3,7,8,9
4,7,18,9
5,10,11,12
7,13,14,15
8,16,17,18


 drop_duplicates no modifica el dataFrame a menos que pongamos inplace = True

In [47]:
df = df.reset_index(drop=True)
df

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9
3,7,18,9
4,10,11,12
5,13,14,15
6,16,17,18


In [49]:
df.iloc[[2,3]]

,A,B,C
2,7,8,9
3,7,18,9


Como vemos aqui son iguales pero en la columna A  y C por lo que podemos remover duplicados de A y C 

In [50]:
df.drop_duplicates(subset=['A','C'], keep='last', inplace=True)
                   # Duplicados en A  y C , nos quedamos con el ultimo valor duplicado , reemplazamos en el df original

In [51]:
df

,A,B,C
0,1,2,3
1,4,5,6
3,7,18,9
4,10,11,12
5,13,14,15
6,16,17,18


### Normalizando columnas

La normalización es una técnica muy usado en la limpieza de datos, normalizar no es más que poner todos los valores en una escala común, sin modificar la diferencia entre los rangos de los valores.

La normalización es crucial en algunos algoritmos para modelar la data correctamente, puesto que a veces tenemos una columna con un valor de 0 a 1 y otra columna con valores entre 40 000 y 50 000, esto conlleva a problemas a la hora de modelar.

In [52]:
from sklearn import preprocessing

In [69]:
df = pd.read_csv('NormalizeColumns.csv')
x = df.values.astype(float)
x

array([[1000.,    2.,    3.],
       [ 400.,    5.,    6.],
       [ 700.,    6.,    9.],
       [ 100.,   11.,   12.],
       [1300.,   14.,   15.],
       [1600.,   17.,   18.]])

Se transformaron los valores del csv a float

In [70]:
min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x)
x_scaled

array([[0.6       , 0.        , 0.        ],
       [0.2       , 0.2       , 0.2       ],
       [0.4       , 0.26666667, 0.4       ],
       [0.        , 0.6       , 0.6       ],
       [0.8       , 0.8       , 0.8       ],
       [1.        , 1.        , 1.        ]])

Se escaló (normalizó) estos valores gracias a preprocessing en específico gracias a MinMaxScaler

Ahora creamos un dataFrame con estos datos normalizados

In [71]:
df = pd.DataFrame(x_scaled, columns = df.columns)
df

,A,B,C
0,0.6,0.000000,0.0
1,0.2,0.200000,0.2
2,0.4,0.266667,0.4
3,0.0,0.600000,0.6
4,0.8,0.800000,0.8
5,1.0,1.000000,1.0


In [64]:
df.columns

RangeIndex(start=0, stop=3, step=1)

### Removiendo valores atípicos

Es usual que en una recolección de datos tengamos valores que se hayan filtrado por error o que hayan sido capturados de forma errada por lo que no representan un valor valuable para nuestro dataset, estos valores tendrán que ser retirados para que el resto ajuste de forma correcta nuestro modelo.

Existen varias maneras de remover estos valores atípicos, aquí veremos dos tipos:
   * Prueba de Tukey
   * Puntaje Z

### Prueba de Tukey

Se basa en el rango Intercuartilico (IQR en inglés) que es la diferencia entre el primer y tercer cuartil de un conjunto de valores. El primer cuartil denotado Q1, es el valor del conjunto de datos que contiene el 25% de los datos debajo de él, y el tercer cuartil denotado Q3, es el valor del conjunto de datos que mantiene al 25% de los datos por encima de él.

   * IQR = Q3-Q1

![](iqr.png)

Luego en la prueba de Tukey los valores atípicos son considerados si:

   * Son menores que Q1 - (1.5 * IQR) 
   * Son mayores que Q3 + (1.5 * IQR)


In [72]:
import numpy as np

In [73]:
def outliers_iqr(data):
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 -q1
    lower_bound = q1 - (1.5*iqr)
    upper_bound = q3 + (1.5*iqr)
    return np.where((data > upper_bound) | (data<lower_bound)) # retorna la locación de items que cumplen la condición

Pondremos aprueba esta función

In [74]:
df = pd.read_csv("http://www.mosaic-web.org/go/datasets/galton.csv")
df.head()

,family,father,mother,sex,height,nkids
0,1,78.5,67.0,M,73.2,4
1,1,78.5,67.0,F,69.2,4
2,1,78.5,67.0,F,69.0,4
3,1,78.5,67.0,F,69.0,4
4,2,75.5,66.5,M,73.5,4


Los valores atípicos de la columna height:

In [89]:
print("Valores atípicos usando funcion outliers_irq")
print("============================================\n")
for i in outliers_iqr(df.height)[0]:
    print(df[i:i+1]) # tambien podemos usar df.iloc[i] pero la representación saldria por columna...

Valores atípicos usando funcion outliers_irq

    family  father  mother sex  height  nkids
288     72    70.0    65.0   M    79.0      7


Observar como solo el valor 288 no será considerado.

In [80]:
outliers_iqr(df.height)[0] 

array([288])

### Puntaje Z

El segundo método para determinar valores atípicos es el método puntaje Z. Un puntaje Z indica la distancia de la media en una medida particular: **variación estandar**, en pocas palabras a cuantas variaciónes estandar esta un dato de la media.

Si tenemos una media u y una desviación estandar o, entonces el puntaje Z de un dato es:

![](zscore.png)

Así interpretaremos un puntaje Z:

   * Un Z < 0 indica que el dato es menos que la media, un Z > 0 indica que el dato es más que la media (algo obvio).
   * Un Z = 0 indica que el dato está al medio, y un Z = 1 nos dice que esta a una desviación estandar de la media y asi...
   * Un Z > 3 o Z < 3 está considerado como valor atípico.
   

In [90]:
def outliers_z_score(data):
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    z_scores = [(y-mean)/std for y in data] # los puntajes z de cada valor en el dato
    return np.where(np.abs(z_scores) > threshold) # nos da la posición de los datos que cumplen la condición interior

In [92]:
print("Valores atípicos usando puntaje Z")
print("=================================\n")
for i in outliers_z_score(df.height)[0]:
    print(df[i:i+1])

Valores atípicos usando puntaje Z

    family  father  mother sex  height  nkids
125     35    71.0    69.0   M    78.0      5
    family  father  mother sex  height  nkids
288     72    70.0    65.0   M    79.0      7
    family  father  mother sex  height  nkids
672    155    68.0    60.0   F    56.0      7


Esta vez obtuvimos 3 valores más, pero si se obtuvo de nuevo el valor 288 que se halló por Prueba de Tukey.

### Resumen

Hemos resuelto un problema de regresión con ayuda de la librería Scikit-Learn, también hemos aprendido a como obntener datasets, generados por nosotros mismos, realizar una limpieza de datos y dos técnicas para ello.
En los siguientes capítulos se verá más algoritmos de machine learning para usarlos para resolver problemas de la vida real...